In [29]:
import torch
import glob
from bounding_box import BoundingBox
import cv2

In [2]:
model = torch.hub.load('yolov5', 'custom', 
    path='./yolov5/runs/train/exp30/weights/best.pt', 
    source='local')

YOLOv5 🚀 2023-2-26 Python-3.10.10 torch-2.0.0+cu117 CUDA:0 (NVIDIA A40, 45494MiB)

Fusing layers... 
Model summary: 267 layers, 46124433 parameters, 0 gradients, 107.7 GFLOPs
Adding AutoShape... 


In [3]:
img_paths = glob.glob("data/output/val/images/*.png")
img_paths[:5]

['data/output/val/images/7_231.png',
 'data/output/val/images/9_28.png',
 'data/output/val/images/0_115.png',
 'data/output/val/images/6_20.png',
 'data/output/val/images/5_117.png']

In [25]:
def getXY(xy):
    plates = xy[xy['name'].str.contains("Plate")]
    if(len(plates) > 0):
        plates = plates.reset_index()
        plates["xCenter"] = plates['xmin'] + plates['xmax'] / 2
        plates["yCenter"] = plates['ymin'] + plates['ymax'] / 2
        plates["width"] = plates['xmax'] - plates['xmin']
        plates["height"] = plates['ymax'] - plates['ymin']
        return [bbox for bbox in plates.apply(lambda x: BoundingBox(*x[['xCenter', 'yCenter', 'width', 'height']]), axis=1)]
    else:
        return None

In [28]:
bboxes = getXY(model(img_paths[1], size=640).pandas().xyxy[0])
bboxes

[Center: (399.1624450683594, 189.15540313720703
       	Width: 41.681640625 px
         Height: 36.69276428222656 px
         Initalized at: time.struct_time(tm_year=2023, tm_mon=4, tm_mday=4, tm_hour=14, tm_min=48, tm_sec=48, tm_wday=1, tm_yday=94, tm_isdst=1),
 Center: (305.9253234863281, 189.3949203491211
       	Width: 16.13787841796875 px
         Height: 30.336288452148438 px
         Initalized at: time.struct_time(tm_year=2023, tm_mon=4, tm_mday=4, tm_hour=14, tm_min=48, tm_sec=48, tm_wday=1, tm_yday=94, tm_isdst=1)]

In [32]:
cam_feed = cv2.VideoCapture(0)
cam_feed.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cam_feed.set(cv2.CAP_PROP_FRAME_HEIGHT, 750)

[ WARN:0@632.511] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ WARN:0@632.511] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_v4l.cpp (902) open VIDEOIO(V4L2:/dev/video0): can't open camera by index


False